In [8]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
dataset = ee.ImageCollection('ESA/WorldCereal/2021/MODELS/v100')


# Function to mask the "other" class (value 0)
def mask_other(img):
    return img.updateMask(img.neq(0))


# Apply mask_other function to the collection
dataset = dataset.map(mask_other)

# Create global mosaic for temporary crops
temporarycrops = dataset.filter(ee.Filter.eq('product', 'temporarycrops')).select('classification').mosaic()
non_cropland_mask = temporarycrops.unmask(1).eq(1)

# Plantation mask
planted = ee.Image("users/duzhenrong/SDPT/sdpt")
plantation_mask = planted.eq(0)

# Night light mask
night_lights = (
    ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')
    .filterDate('2022-01-01', '2022-12-31')
    .select('avg_rad')
    .mean()
)
night_light_threshold = 2
night_light_mask = night_lights.lt(night_light_threshold)

# Land cover mask
world_cover = ee.Image('ESA/WorldCover/v100/2020')
crop_mask = (
    world_cover.neq(40)
    .And(world_cover.neq(50))
    .And(world_cover.neq(70))
    .And(world_cover.neq(80))
)

# Water mask
land_water_mask = ee.Image("MODIS/MOD44W/MOD44W_005_2000_02_24").select('water_mask')
land_mask = land_water_mask.eq(0)


# NDVI calculation function
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    ndvi_non_negative = ndvi.where(ndvi.lt(0), 0)
    return image.addBands(ndvi_non_negative)


# Albedo approximation function (using LST band)
def calculate_albedo(image):
    lst = image.select('ST_B10')
    albedo = lst.rename('Albedo')
    return image.addBands(albedo)


# Load protected areas mask
protected_areas = ee.FeatureCollection('WCMC/WDPA/current/polygons')


def buffer_feature(feature):
    return feature


protected_areas = protected_areas.map(buffer_feature)
protected_mask = ee.Image.constant(1).clip(protected_areas).rename('Protected')


# Sample collection function
def sample_class(continent_geometry, land_class, cls, sample_num):
    mask_cluster = land_class.eq(cls)
    landsat_collection = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterDate('2021-01-01', '2022-12-31')
        .filter(ee.Filter.lt('CLOUD_COVER', 10))
        .filterBounds(mask_cluster.geometry())
    )
    landsat_with_indices = landsat_collection.map(calculate_ndvi)
    median_image = landsat_with_indices.median()
    ndvi_image = median_image.select(['NDVI'])
    masked_image = ndvi_image.updateMask(land_mask).clip(continent_geometry)

    composite = masked_image.updateMask(protected_mask)
    composite = (
        composite.updateMask(night_light_mask)
        .updateMask(crop_mask)
        .updateMask(non_cropland_mask)
        .updateMask(mask_cluster)
    )
    normalized_image = composite.subtract(0.1).divide(0.5 - 0.1).clamp(0, 1).clip(composite.geometry())
    ndvi_bins = normalized_image.select('NDVI').multiply(10).int()
    stratification = ndvi_bins.reduce(ee.Reducer.sum()).rename('sum')

    stratified_sample = stratification.stratifiedSample(
        numPoints=sample_num,
        classBand='sum',
        region=stratification.geometry(),
        scale=100,
        seed=1935,
        dropNulls=True,
        geometries=True,
        tileScale=16
    )
    return stratified_sample


# Continent geometries
continents = {
    'Africa': ee.Geometry.Polygon([[-18.0, 38.0], [52.0, 38.0],
                                   [52.0, -35.0], [-18.0, -35.0]]),
    'AsiaW': ee.Geometry.Polygon([[23.0, 81.0], [100.0, 81.0],
                                  [100.0, 0.0], [23.0, 0.0]]),
    'AsiaE': ee.Geometry.Polygon([[100.0, 81.0], [180.0, 81.0],
                                  [180.0, 0.0], [100.0, 0.0]]),
    'Europe': ee.Geometry.Polygon([[-25.0, 72.0], [45.0, 72.0],
                                   [45.0, 35.0], [-25.0, 35.0]]),
    'NorthAmericaW': ee.Geometry.Polygon([[-180.0, 84.0], [-100.0, 84.0],
                                          [-100.0, 7.0], [-180.0, 7.0]]),
    'NorthAmericaE': ee.Geometry.Polygon([[-100.0, 84.0], [-30.0, 84.0],
                                          [-30.0, 7.0], [-100.0, 7.0]]),
    'SouthAmerica': ee.Geometry.Polygon([[-82.0, 13.0], [-30.0, 13.0],
                                         [-30.0, -60.0], [-82.0, -60.0]]),
    'Oceania': ee.Geometry.Polygon([[110.0, 0.0], [180.0, 0.0],
                                    [180.0, -50.0], [110.0, -50.0]])
}

ccList = ('Africa',
          'AsiaW',
          'AsiaE',
          'Europe',
          'NorthAmericaW',
          'NorthAmericaE',
          'SouthAmerica',
          'Oceania')

In [7]:
# Sampling
# create a list of cluster numbers
list1 = list(range(0, 3000, 100))
list2 = list(range(0, 6))
result = [a + b for a in list1 for b in list2]

for cc in list(range(0, 8)):
   
    # print(continents[cc])
    cN = ccList[cc]

    geometry = continents[cN];

    land_class_map = ee.Image(
        f'projects/landsat-leafn/assets/Cluster_L2_V1_{cN}')

    for cln in result:
        sample_con = sample_class(geometry, land_class_map, cln, 100)
        
        # # Export the sample
        task = ee.batch.Export.table.toDrive(
            collection=sample_con,
            description=f'Sample_L2_{cN}_{cln}',
            folder='Sample_good_L2',
            fileNamePrefix=f'Sample_L2_{cN}_{cln}',
            fileFormat='CSV'
            )
        task.start()